In [1]:
import pandas as pd
import numpy as np
import math as mt
import matplotlib.pyplot as plt
import seaborn as sns
%autosave 5

Autosaving every 5 seconds


In [2]:
df = pd.read_excel("default_of_credit_card_clients.xls")
df.shape

(30000, 25)

In [3]:
df.isnull().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_1                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64

In [4]:
df.ID.value_counts().max()

2

In [5]:
df.shape[0]-df.ID.nunique()

313

In [6]:
id_counts = df.ID.value_counts()

In [7]:
dups = id_counts == 2
dups

90f662f9-c1e3     True
9929aa3b-d8f6     True
c89b26ff-ec7f     True
4a051fdc-d04c     True
94d59251-fa7f     True
                 ...  
41d14dfd-e6f8    False
532d669b-8cfc    False
b51bcf84-3bcb    False
438cea9f-af68    False
8ac5e198-db93    False
Name: ID, Length: 29687, dtype: bool

In [8]:
mask = id_counts[dups].index.tolist()

In [9]:
df.loc[df.ID.isin(mask[:5]),:]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
3396,9929aa3b-d8f6,210000,2,1,2,43,Not available,-1,0,0,...,139784,142662,145725,184212,5482,3493,3590,3815,3876,0
3496,9929aa3b-d8f6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8122,94d59251-fa7f,130000,1,2,2,30,Not available,0,0,0,...,131743,131986,131430,4800,5250,5050,5700,5100,5012,0
8222,94d59251-fa7f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12898,90f662f9-c1e3,50000,1,2,2,31,Not available,0,0,0,...,4011,1000,0,2000,1500,1000,1000,0,0,0
12998,90f662f9-c1e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21056,4a051fdc-d04c,100000,1,1,2,34,0,0,2,2,...,26839,26109,27686,7100,0,3000,0,2000,0,1
21156,4a051fdc-d04c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24265,c89b26ff-ec7f,370000,1,1,2,31,0,0,0,0,...,44178,47838,54341,5000,3000,5000,5000,8000,10000,0
24365,c89b26ff-ec7f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
temp = df == 0
temp.iloc[:,1:].all(axis=1).sum() # Values with all the rows zeros

315

In [11]:
zero_mask = temp.iloc[:,1:].all(axis=1)

In [12]:
df = df[~zero_mask]

In [13]:
30000-df.shape[0]

315

In [14]:
df.shape[0]-df['ID'].nunique()

0

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29685 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          29685 non-null  object
 1   LIMIT_BAL                   29685 non-null  int64 
 2   SEX                         29685 non-null  int64 
 3   EDUCATION                   29685 non-null  int64 
 4   MARRIAGE                    29685 non-null  int64 
 5   AGE                         29685 non-null  int64 
 6   PAY_1                       29685 non-null  object
 7   PAY_2                       29685 non-null  int64 
 8   PAY_3                       29685 non-null  int64 
 9   PAY_4                       29685 non-null  int64 
 10  PAY_5                       29685 non-null  int64 
 11  PAY_6                       29685 non-null  int64 
 12  BILL_AMT1                   29685 non-null  int64 
 13  BILL_AMT2                   29685 non-null  in

In [16]:
df['PAY_1'].value_counts().index.tolist()

[0, -1, 1, 'Not available', -2, 2, 3, 4, 5, 8, 6, 7]

In [17]:
sum(df['PAY_1'] != 'Not available')

26664

In [18]:
df['PAY_1'].replace(to_replace='Not available',value=np.nan,inplace=True)
df['PAY_1'].value_counts()

 0.0    13087
-1.0     5047
 1.0     3261
-2.0     2476
 2.0     2378
 3.0      292
 4.0       63
 5.0       23
 8.0       17
 6.0       11
 7.0        9
Name: PAY_1, dtype: int64

**Education: 1=graduate school; 2=university; 3=high school; 4=others**

In [19]:
df['EDUCATION'].value_counts()

2    13884
1    10474
3     4867
5      275
4      122
6       49
0       14
Name: EDUCATION, dtype: int64

In [20]:
df['EDUCATION'].replace(to_replace=[0,5,6],value=4,inplace=True)
df['EDUCATION'].value_counts()

2    13884
1    10474
3     4867
4      460
Name: EDUCATION, dtype: int64

**SEX : Gender (1 = male; 2 = female)**

In [21]:
df['SEX'].value_counts()

2    17910
1    11775
Name: SEX, dtype: int64

**MARRIAGE : Marital status (1 = married; 2 = single; 3 = others)**

In [22]:
df['MARRIAGE'].value_counts()

2    15810
1    13503
3      318
0       54
Name: MARRIAGE, dtype: int64

In [23]:
df['MARRIAGE'].replace(to_replace=0,value=3,inplace=True)
df['MARRIAGE'].value_counts()

2    15810
1    13503
3      372
Name: MARRIAGE, dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29685 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          29685 non-null  object 
 1   LIMIT_BAL                   29685 non-null  int64  
 2   SEX                         29685 non-null  int64  
 3   EDUCATION                   29685 non-null  int64  
 4   MARRIAGE                    29685 non-null  int64  
 5   AGE                         29685 non-null  int64  
 6   PAY_1                       26664 non-null  float64
 7   PAY_2                       29685 non-null  int64  
 8   PAY_3                       29685 non-null  int64  
 9   PAY_4                       29685 non-null  int64  
 10  PAY_5                       29685 non-null  int64  
 11  PAY_6                       29685 non-null  int64  
 12  BILL_AMT1                   29685 non-null  int64  
 13  BILL_AMT2                   296

PAY_1–Pay_6 : A record of past payments. Past monthly payments, recorded from April to
September, are stored in these columns.  
PAY_1 represents the repayment status in September; PAY_2 = repayment status in August; and
so on up to PAY_6 , which represents the repayment status in April.  
The measurement scale for the repayment status is as follows: -1 = pay duly; 1 = payment delay for
one month; 2 = payment delay for two months; and so on up to 8 = payment delay for eight months;
9 = payment delay for nine months and above.

BILL_AMT1 represents the bill statement amount in September; BILL_AMT2 represents the bill
statement amount in August; and so on up to BILL_AMT7 , which represents the bill statement
amount in April.

PAY_AMT1 – PAY_AMT6 : Amount of previous payment (NT dollar). PAY_AMT1 represents the
amount paid in September; PAY_AMT2 represents the amount paid in August; and so on up to
PAY_AMT6 , which represents the amount paid in April.

In [25]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

In [26]:
df.to_csv('cleaned_data.csv',index=False)

In [27]:
df = pd.read_csv('cleaned_data.csv')
df.shape

(29685, 25)